In [ ]:
PROJECT_DIR = ''
#APP_SYS_NAME = 'BGL'
APP_SYS_NAME = 'Thunderbird'

In [ ]:
BASE_DIR = PROJECT_DIR + 'output/'
RAW_DIR = PROJECT_DIR + 'raw_files/'

# Imports

In [ ]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, confusion_matrix , precision_score, recall_score, f1_score
from sklearn.metrics import completeness_score, homogeneity_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay
from scipy.spatial.distance import cosine
import warnings
import matplotlib.pyplot as plt
import sys
import subprocess
import random

random.seed(2)
sys.path.append(PROJECT_DIR) # this is done to make the import of ad_feature_extraction work
from ad_feature_extraction import parsers

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from matplotlib.lines import Line2D

# Preprocessing

In [ ]:
input_data = pd.read_csv(BASE_DIR + APP_SYS_NAME + '_clusters2.csv')

print("Input_data Shape:",input_data.shape)
print(input_data['label'].value_counts())

sns.countplot(x=input_data['label'])
plt.show()

print(input_data[['clusters', 'label', 'text']].groupby(['clusters', 'label']).count())
print(f"Homogeneity Score: \n {homogeneity_score(input_data['label'], input_data['clusters'])}\n")
print(input_data[['cluster2', 'label', 'text']].groupby(['cluster2', 'label']).count())

In [ ]:
le = LabelEncoder()
input_data['label'] = le.fit_transform(input_data['label']) # false:0 and true:1

X = input_data[['tfidf_text']].dropna()
y = input_data.loc[X.index,'label'].values

print("columns for the X file"+ str(X.columns))
print(X.shape,y.shape)

In [ ]:
# do tfidf and kmeans on input_data and get homegeneity and completeness scores.
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X['tfidf_text'])
print('tfidf shape', X_tfidf.shape)

k = input_data['cluster2'].max() + 1
print('Using k = ', k)
kmeans = MiniBatchKMeans(n_clusters=k, n_init=10, batch_size=10000, random_state=0)
# KMeans(k, random_state=0)
kmeans.fit(X_tfidf)

pred = kmeans.predict(X_tfidf)
labels = y

print("_______________________________________________")
print(f"Completeness Score: \n {completeness_score(labels, pred)}\n")
print(f"Homogeneity Score: \n {homogeneity_score(labels, pred)}\n") 

In [ ]:
# count log lines within each cluster
output = X.copy()
output['cluster2'] = pred
output['label'] = input_data.loc[X.index, 'label']
print(output.groupby(['cluster2', 'label']).count())

# Use label to determine which clusters are anomalies

In [ ]:
print(output.groupby(['cluster2', 'label']).count())
counts_by_label_df = output.groupby(['cluster2', 'label']).count().reset_index()
pos_df = counts_by_label_df[counts_by_label_df['label'] == 1]
neg_df = counts_by_label_df[counts_by_label_df['label'] == 0]
anomaly_clusters = []
normal_clusters = []
for _, row in pos_df.iterrows():
    cluster = row['cluster2']
    pos = row['tfidf_text']
    neg = neg_df[neg_df["cluster2"] == cluster]['tfidf_text'].sum()
    print(cluster, 'pos neg', pos, neg)
    if pos > neg:
        anomaly_clusters.append(cluster)

for _, row in neg_df.iterrows():
    cluster = row['cluster2']
    neg = row['tfidf_text']
    pos = pos_df[pos_df["cluster2"] == cluster]['tfidf_text'].sum()
    if neg >= pos:
        normal_clusters.append(cluster)

print('anomaly clusters', anomaly_clusters)
print('normal clusters', normal_clusters)

# Create summary report for clusters. Give example log line for each cluster.

In [ ]:
summary_file_name = BASE_DIR + APP_SYS_NAME + '_cluster_summary.csv'
output_file = open(summary_file_name, 'w')
with output_file:
    writer = csv.writer(output_file)
    writer.writerow(['is_anomaly', 'anomaly_count', 'normal_count',
                     'cluster', 'pct_of_total',
                     'most_similar_sample', 'most_similar_distance',
                     'least_similar_sample', 'least_similar_distance'])
    
    def print_clusters(clusters, label):
        num_rows = X.shape[0]
        for ac in clusters:
            indices = X.index[pred == ac]
            rows_for_cluster = X_tfidf[pred == ac]
            centroid = kmeans.cluster_centers_[ac]
            least_distance = 0
            most_similar = None
            greatest_distance = 1
            least_similar = None
            for i in range(rows_for_cluster.shape[0]):
                row = np.asarray(rows_for_cluster[i].todense()).reshape(-1)
                distance = cosine(centroid, row)
                if most_similar is None or least_distance > distance:
                    least_distance = distance
                    most_similar = i
        
                if least_similar is None or greatest_distance < distance:
                    greatest_distance = distance
                    least_similar = i
                
            pct = round(np.count_nonzero(pred == ac) / num_rows, 2)
            writer.writerow([1 if label == 'anomaly' else 0,
                             pos_df[pos_df["cluster2"] == ac]['tfidf_text'].sum(),
                             neg_df[neg_df["cluster2"] == ac]['tfidf_text'].sum(),
                             ac, pct,
                             input_data.loc[indices[most_similar], 'text'], round(least_distance, 3),                  
                             input_data.loc[indices[least_similar], 'text'], round(greatest_distance, 3)])
    print_clusters(anomaly_clusters, 'anomaly')
    print_clusters(normal_clusters,  ' normal')
    
summary_df = pd.read_csv(summary_file_name)
summary_df

# Mispredictions

In [ ]:
for ac in anomaly_clusters:
    indices = input_data.index[(pred == ac) & (input_data['label'] == 0)]
    if indices.size == 0:
        continue
    print('False Positives for cluster', ac, 'count', indices.size)
    print(np.unique(input_data.loc[indices, 'text'].values))
    print(np.unique(input_data.loc[indices, 'tfidf_text'].values))

for nc in normal_clusters:
    indices = input_data.index[(pred == nc) & (input_data['label'] == 1)]
    if indices.size == 0:
        continue
    print('False Negatives for cluster', nc, 'count', indices.size)
    print(np.unique(input_data.loc[indices, 'text'].values))
    print(np.unique(input_data.loc[indices, 'tfidf_text'].values))

# Use TFIDF and KMeans from above to predict anomalies in original raw file

In [ ]:
cluster_to_label_map = np.zeros(k)
for c in anomaly_clusters:
    cluster_to_label_map[c] = 1
    
for c in normal_clusters:
    cluster_to_label_map[c] = 0

mispredictions_file_name = BASE_DIR + APP_SYS_NAME + '_mispredictions.csv'
output_file = open(summary_file_name, 'w')

def process_raw_file(raw_filename, parser):
    block_size = 100000 # entire dataset will not fit in memory, so do a block at a time
    file_path = raw_filename.split('/')
    with open(raw_filename, "r", encoding="utf8", errors='ignore') as raw_file:
        count = 0
        fp_total = tp_total = fn_total = 0
        row_block = []
        y_block = np.zeros(block_size)
        for line in raw_file:
            line = line.rstrip('\n')
            epochts,text,is_anomaly,filename = parser(file_path, line)
            y_block[count % block_size] = 1 if is_anomaly else 0
            count += 1
            row_block.append(text)
            
            if count % block_size == 0:
                X_block = tfidf.transform(row_block)
                block_clusters = kmeans.predict(X_block)
                block_pred = np.zeros(block_size)
                for i in range(block_clusters.shape[0]):
                    block_pred[i] = cluster_to_label_map[block_clusters[i]]
                        
                fp = (block_pred == 1) & (y_block == 0)
                tp = (block_pred == 1) & (y_block == 1)
                fn = (block_pred == 0) & (y_block == 1)
                fp_total += np.count_nonzero(fp)
                tp_total += np.count_nonzero(tp)
                fn_total += np.count_nonzero(fn)
                print(count, fp_total, tp_total, fn_total)
                row_block = []

            # Remove this condition to process full file.
            if count > 3000000: break
        print(raw_filename, 'precision', tp_total / (fp_total + tp_total))
        print(raw_filename, 'recall', tp_total / (fn_total + tp_total))

process_raw_file(RAW_DIR + APP_SYS_NAME + '/' + APP_SYS_NAME + '.log', parsers[APP_SYS_NAME])

# Compute precision, recall, homogeneity, confusion matrix

In [ ]:
def report(report_name, pred, labels):
    label_pred = np.zeros(pred.size)
    for i in range(pred.size):
        label_pred[i] = cluster_to_label_map[pred[i]]
    kmeans_report = pd.DataFrame(classification_report(labels, label_pred, output_dict=True))
    print(f"{report_name} Result:\n================================================")        
    print(f"Accuracy Score: {accuracy_score(labels, label_pred) * 100:.2f}%")
    print("_______________________________________________")
    print(f"CLASSIFICATION REPORT:\n{kmeans_report}")
    print("_______________________________________________")
    print(f"Confusion Matrix: \n {confusion_matrix(labels, label_pred)}\n")
    
    print("_______________________________________________")
    print(f"Completeness Score: \n {completeness_score(labels, pred)}\n")
    print("_______________________________________________")
    print(f"Homogeneity Score: \n {homogeneity_score(labels, pred)}\n")
    print("_______________________________________________")
    print(f'F1 Score: \n {f1_score(labels, label_pred)}')
    print("_______________________________________________")
    print(f'Precision Score: \n {precision_score(labels, label_pred)}')
    print("_______________________________________________")
    print(f'Recall Score: \n {recall_score(labels, label_pred)}')
    print("_______________________________________________")
    print(f'Roc AUC Score: \n {roc_auc_score(labels, label_pred)}')
   
    ConfusionMatrixDisplay.from_predictions(labels, label_pred)
    plt.show()
    
    PrecisionRecallDisplay.from_predictions(labels, label_pred, name=report_name)
    plt.show()

report(APP_SYS_NAME, pred, output.label.values)

# TSNE Scatterplot

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

tfidf_tsne = TfidfVectorizer()
X_tfidf_tsne = tfidf_tsne.fit_transform(input_data.loc[X.index, 'text'])

print(output.shape[0])
output_sample = output.sample(1500, random_state=0)
X_tfidf_sample = X_tfidf_tsne[output_sample.index]

for perplexity in range(35, 36, 5):
    tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity, n_iter=1000, learning_rate='auto', random_state=1, init="random")
    tsne_result = tsne.fit_transform(np.asarray(X_tfidf_sample.todense()))
    plt.figure(figsize=(8,5))


    tsne_df = output_sample.copy()
    tsne_df['tsne_one'] = tsne_result[:,0]
    tsne_df['tsne_two'] = tsne_result[:,1]

    ax= sns.scatterplot(
        x='tsne_one', y='tsne_two',
        hue="cluster2",
        palette=sns.color_palette("hls", 20),
        #palette=sns.color_palette("blend:white,white"),
        data=tsne_df,
        legend=None,
        alpha=1,
    )
    ax.set(xlabel=None, ylabel=None)
    ax.tick_params(labelleft=False, left=False, labelbottom=False, bottom=False)
    ax.set_frame_on(False)

    label_df = tsne_df[tsne_df['label'] == 1]
    
    legend_elements = [Line2D([0], [0], marker=u'$\u25CC$', color='white', markeredgecolor='black', label='True Positive', markerfacecolor='white', markersize=10, alpha=1),
                       Line2D([0], [0], marker='o', color='white', markeredgecolor='red', label='False Positive', markerfacecolor='white', markersize=10),
                       Line2D([0], [0], marker='o', color='white', markeredgecolor='green', label='False Negative', markerfacecolor='white', markersize=10)]
    ax.legend(handles=legend_elements, loc='lower right')

    tsne_centers = np.zeros((kmeans.cluster_centers_.shape[0], 2))
    tsne_radius = np.zeros(kmeans.cluster_centers_.shape[0])
    for i in range(kmeans.cluster_centers_.shape[0]):
        tsne_cluster = tsne_result[output_sample.cluster2 == i]
        if tsne_cluster.sum() == 0: continue
        tsne_centers[i] = np.mean(tsne_cluster, axis=0)
        tsne_radius[i] = np.max(np.max(tsne_cluster, axis=0) - np.min(tsne_cluster, axis=0))
    
    anomaly_tsne_centers = tsne_centers[anomaly_clusters]
    anomaly_tsne_radius = tsne_radius[anomaly_clusters]
    plt.scatter(anomaly_tsne_centers[:, 0], anomaly_tsne_centers[:, 1], s=anomaly_tsne_radius*200,
                facecolors='none', edgecolors='black', linestyle='dashed')
    
    false_negatives = tsne_result[output_sample.cluster2.isin(normal_clusters) & (output_sample.label == 1)]
    plt.scatter(false_negatives[:, 0], false_negatives[:, 1], s=100, facecolors='none', edgecolors='green')
    
    false_positives = tsne_result[output_sample.cluster2.isin(anomaly_clusters) & (output_sample.label == 0)]
    plt.scatter(false_positives[:, 0], false_positives[:, 1], s=100, facecolors='none', edgecolors='red')
    
    
    plt.show()


# Predictability

In [ ]:
window_df = pd.DataFrame()
cluster_seq = input_data[['cluster2']].values
#cluster_seq = np.arange(50)
y_df = np.squeeze(cluster_seq)

print('uniq clusters', np.unique(y_df))

for w in range(1, 6):
    if w == 0:
        continue
    col = "w+" + str(w)
    padding = np.ones((1, w)) * -1
    window_df[col] = np.append(cluster_seq[w:], padding)

n = len(cluster_seq)
for w in range(1, 6):
    if w == 0:
        continue
    col = "w-" + str(w)
    padding = np.ones((1, w)) * -1
    window_df[col] = np.append(padding, cluster_seq[:n-w])

    
print(window_df.head(10))
print('window', window_df.shape, 'y_df', y_df.shape)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(window_df, y_df)
print('classes', gnb.classes_)

idx = np.ones(gnb.classes_.max() + 1).astype(int) * -1
for i, c in enumerate(gnb.classes_):
    idx[c] = i

print('idx', idx)

probs = gnb.predict_proba(window_df)
pred_probs = np.zeros(probs.shape[0])
for i in range(probs.shape[0]):
    col = idx[y_df[i]]
    pred_probs[i] = probs[i, col]


prob_totals = np.zeros(probs.shape[1])
prob_counts = np.zeros(probs.shape[1])
for i in range(probs.shape[1]):
    cls = gnb.classes_[i]
    prob_totals[i] = pred_probs[y_df == cls].sum()
    prob_counts[i] = np.count_nonzero(y_df == cls)
    #print('probs', i, cls, prob_totals[i], prob_counts[i])

predictiveness = prob_totals / prob_counts
print(predictiveness)
print(np.argsort(predictiveness))


for i in anomaly_clusters:
    print('anomaly cluster', i, 'predictiveness', predictiveness[idx[i]])

for i in normal_clusters:
    print('normal cluster', i, 'predictiveness', predictiveness[idx[i]])


In [ ]:
from scipy.stats import kurtosis, skew

num_clusters = input_data['cluster2'].max() + 1
stats_df = pd.DataFrame({'cluster': [j for j in range(num_clusters)]})

kurt_results = []
skew_results = []

offset = 0
block_size = 10000
while offset < len(input_data):
    block_df = input_data.iloc[offset:offset+block_size]
    mean_timestamp = block_df['timestamp'].mean()
    offset += block_size
    for k in range(num_clusters):
        block = block_df[block_df['cluster2'] == k]['timestamp'].values - mean_timestamp
        if len(block) == 0:
            b_kurt = b_skew = 0
        else:
            b_kurt = kurtosis(block)
            b_skew = skew(block)
        kurt_results.append((b_kurt, k))
        skew_results.append((b_skew, k))

kurt_results = sorted(kurt_results, reverse=True)
skew_results = sorted(skew_results, reverse=True)


min_timestamp = input_data['timestamp'].min()

def cluster_dist(k):
    result = input_data[input_data['cluster2'] == k]['timestamp'] - min_timestamp
    if result.size == 0:
        result = input_data['timestamp'] - min_timestamp
    np_result = np.asarray(result.values)
    mean = np_result.mean()
    std = np_result.std()
    np_result = (np_result - mean) / std
    stats_df.at[k, 'kurtosis'] = kurtosis(np_result)
    stats_df.at[k, 'skew'] = skew(np_result)

for k in range(num_clusters):
    cluster_dist(k)

stats_df.dropna()
print(input_data[['cluster2', 'label', 'clusters']].groupby(['cluster2', 'label']).count())
for col in stats_df.columns:
    print(col, np.argsort(stats_df[col].values))

print(stats_df)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM

random_state=5

for ModelClass in [IsolationForest, SGDOneClassSVM]:
    model = ModelClass(random_state=random_state)
    model_pred = model.fit_predict(X_tfidf)
    model_pred[model_pred == 1] = 0
    model_pred[model_pred == -1] = 1
    model_name = type(model).__name__
    print("_______________________________________________")
    print(f"{model_name}:")
    print("Label counts", np.bincount(model_pred))
    print(f"Completeness Score: \n {completeness_score(model_pred, pred)}\n")
    print(f"Homogeneity Score: \n {homogeneity_score(model_pred, pred)}\n")
    